In [ ]:
import tensorflow as tf
import numpy as np
from data import dataprep
from tensorflow.contrib import layers
from tensorflow.contrib.learn import *
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
from tensorflow.python.estimator.inputs import numpy_io

In [ ]:
train_dict, test_dict = dataprep.mf_train_test()
rating_train = train_dict.pop("rating")
rating_test = test_dict.pop("rating")

In [ ]:
model_params = dict(
    n_user=train_dict['user_id'].max() + 1,
    n_movie=train_dict['movie_id'].max() + 1,
    n_dim=20,
    reg_param=0.01,
    learning_rate=0.01,
)

In [ ]:
def mf_function(features, targets, mode, params):
    users = features['user_id']
    movies = features['movie_id']
    ratings = targets

    with tf.variable_scope("embedding"):
        user_weight = tf.get_variable("user_w"
                                      , shape=[params['n_user'], params['n_dim']]
                                      , dtype=tf.float32
                                      , initializer=layers.xavier_initializer())
        user_bias = tf.get_variable("user_b"
                                    , shape=[params['n_user']]
                                    , dtype=tf.float32
                                    , initializer=tf.zeros_initializer)
        movie_weight = tf.get_variable("movie_w"
                                       , shape=[params['n_movie'], params['n_dim']]
                                       , dtype=tf.float32
                                       , initializer=layers.xavier_initializer())
        movie_bias = tf.get_variable("movie_b"
                                     , shape=[params['n_movie']]
                                     , dtype=tf.float32
                                     , initializer=tf.zeros_initializer)
        for v in tf.trainable_variables():
            tf.summary.histogram(name=v.name.replace(":0",""), values=v)

    with tf.name_scope("inference"):
        user_emb = tf.nn.embedding_lookup(user_weight, users)
        u_b = tf.nn.embedding_lookup(user_bias, users)
        movie_emb = tf.nn.embedding_lookup(movie_weight, movies)
        m_b = tf.nn.embedding_lookup(movie_bias, movies)
        pred = tf.reduce_sum(tf.multiply(user_emb, movie_emb), 1) + u_b + m_b

    with tf.name_scope("loss"):
        reg_loss = layers.apply_regularization(layers.l2_regularizer(scale=params['reg_param']),
                                               weights_list=[user_weight, movie_weight])
        loss = tf.nn.l2_loss(pred - ratings) + reg_loss
        rmse = tf.sqrt(tf.reduce_mean(tf.pow(pred - ratings, 2)))
        

    eval_metric_ops = {'rmse': rmse}
    train_op = layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=params["learning_rate"],
        optimizer=tf.train.AdamOptimizer,
        summaries=[
            "learning_rate",
            "loss",
            "gradients",
            "gradient_norm",
        ])

    return ModelFnOps(mode=mode, predictions=pred, loss=loss,
                      train_op=train_op, eval_metric_ops=eval_metric_ops)

In [ ]:
# input queue for training
train_input_fn = numpy_io.numpy_input_fn(
    x=train_dict, y=rating_train, batch_size=10000, shuffle=True, num_epochs=None)
# input queue for evaluation on test data
test_input_fn = numpy_io.numpy_input_fn(
    x=test_dict, y=rating_test, batch_size=rating_test.shape[0], shuffle=False, num_epochs=None)

monitor_test = monitors.ValidationMonitor(input_fn=test_input_fn, eval_steps=1, every_n_steps=10,
                                                name='test')

monitor_train = monitors.ValidationMonitor(input_fn=train_input_fn, eval_steps=1, every_n_steps=10,
                                                name='train')

mf_estimator = Estimator(
    model_fn=mf_function,
    params=model_params,
    model_dir='_summary/mf_estimator',
    config=RunConfig(save_checkpoints_secs=5))

mf_estimator.fit(input_fn=train_input_fn, steps=400, monitors=[monitor_test, monitor_train])